In [11]:
from sklearn.externals import joblib

In [12]:

clf = joblib.load('classify.model') # 분류기 엔진
cate_dict = joblib.load('cate_dict.dat') # id별 카테고리가 맵핑되어있는 파일
vectorizer = joblib.load('vectorizer.dat') # 제목들을 숫자 id로 변환해주는 것


In [13]:
joblib.dump(clf,'n_classify.model') # 무시해도 됨

['n_classify.model',
 'n_classify.model_01.npy',
 'n_classify.model_02.npy',
 'n_classify.model_03.npy']

In [14]:
joblib.dump(cate_dict,'n_cate_dict.dat')
joblib.dump(vectorizer,'n_vectorizer.dat')

['n_vectorizer.dat']

In [15]:
cate_id_name_dict = dict(map(lambda (k,v):(v,k),cate_dict.items()))

In [16]:
vec = vectorizer.transform(['[신한카드5%할인][서우한복] 아동한복 여자아동 금나래 (분홍)'])
print vec # 0은 0번째 피쳐란 뜻?

  (0, 14270)	1
  (0, 17887)	1
  (0, 18536)	1
  (0, 19597)	1
  (0, 19786)	1
  (0, 20652)	1
  (0, 25237)	1


In [17]:
pred = clf.predict(vec)[0]
print pred
print cate_id_name_dict[pred]

33
패션의류;아동의류;한복


In [18]:
# unigram
d = u'아동한복 여아'
print ";".join(d)

아;동;한;복; ;여;아


In [19]:
# bigram
b = u'아동 동한 한복 여아'

In [10]:
# unigram, bigram, trigram은
# vectorize에서 써도 되고 오픈 소스 가져다 써도 되고 자기가 짜서 써도 되고...

In [ ]:
# set remove list

mapping = { 'A/S':'AS'}
stop_words = [u"%즉시할인쿠폰",  u"5% 즉시할인", u"0% 즉시할인", u"% 즉시할인", u"5% 할인", u"0% 할인", u"5%할인", u"0%할인", u"% 할인", u"적립",
              u"쿠폰", u"즉시",u"정품", u"증정", u"할인",
              u"해외직배송", u"직배송", u"무료배송", u"안전배송", u"당일", u"배송", u"발송", u"빠른",
              u"신한카드", u"BC카드", u"KB국민카드", u"국민카드", u"롯데카드", u"KB카드", u"현대카드", u"삼성카드", u"카드", u"일까지"]
symbols = ["[","]","(",")", "+", "-", "/", "?", "~", "*", u"☆", u"★", u"●", u"♥", u"■", u"▶", u"◀", u"↑"]
ambg_words = [u"해외", u"구매대행"]

start = 0
length = 1
remove_ambg = True

# define concatenate function
def concate(list1, list2) :
    for each in list2:
        list1.append(each)
    return list1

remove_list = concate(stop_words, symbols)
if remove_ambg:
    remove_list = concate(remove_list,ambg_words)

In [ ]:
from bottle import route, run, template,request,get, post


import  time
from threading import  Condition
_CONDITION = Condition()
@route('/classify')
def classify():
    print "classify called"
    img = request.GET.get('img','')
    name = request.GET.get('name', '')
    
    # remove stop words
    # replace
    for before, after in mapping.iteritems():
        name = name.replace(before, after)
    #remove
    for each in remove_list :
        name = name.replace(each," ")
    
    vec = vectorizer.transform([name])
    p = clf.predict(vec)
    pred = p[0]
    return {'cate':cate_id_name_dict[pred]}


run(host='0.0.0.0', port=8887)
# http://192.168.99.100:8887/classify?name=%27ad%27&img=%27asd%27

Bottle v0.12.9 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8887/
Hit Ctrl-C to quit.



classify called
vec
  (0, 22991)	1
p
[6]


192.168.99.1 - - [15/Aug/2016 10:31:49] "GET /classify?name=%EC%B6%95%EA%B5%AC&img=%27asd%27 HTTP/1.1" 200 80
192.168.99.1 - - [15/Aug/2016 10:31:49] "GET /favicon.ico HTTP/1.1" 404 747


classify called
vec
  (0, 265)	1
  (0, 18173)	1
  (0, 19597)	1
  (0, 19786)	1
  (0, 20629)	1
  (0, 20758)	1
  (0, 22418)	1
  (0, 25237)	1
p
[33]


172.17.0.1 - - [15/Aug/2016 10:32:05] "GET /classify?name=[%EC%8B%A0%ED%95%9C%EC%B9%B4%EB%93%9C5%%ED%95%A0%EC%9D%B8][%EC%98%88%ED%99%94-%EC%A2%8B%EC%9D%80%EC%95%84%EC%9D%B4%EB%93%A4]%20%EC%95%84%EB%8F%99%ED%95%9C%EB%B3%B5%20%EC%97%AC%EC%95%84%201076%20%EB%B9%9B%EC%9D%B4%EB%82%98%EB%85%B8%EB%9E%91&img= HTTP/1.1" 200 74


 * 추후 여기 docker 에서 뭔가 python package 설치할게 있으면 
 * /opt/conda/bin/pip2 install bottle 이런식으로 설치 가능